# README
해당 작업은 관련 작업내용을 진행하던 중 랭그래프에 대한 내용을 인지. 

질문에 따른 적절한 GPT 처리 흐름(역할)을 선택해서 사용하는 것에 대해서 기술 관심이 생김

해당 내용에 대한 간단한 예시를 이용해보려고 함

In [5]:
from langchain_ollama.chat_models import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory
# from langchain_ollama.chat_models import ChatOpenAI
# from langchain.schema import Sys
from langgraph.graph import StateGraph, END
# .graph import StateGraph, END

In [2]:
key = open('../../../api_key.txt','r')
api_key = key.read()
# openai.api_key = api_key

base_ = open('../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

In [3]:
# 상태 정의 (우리는 간단히 dict 씀)
state_type = dict

In [ ]:
 = OllamaEmbeddings(model = 'bge-m3',
                 base_url=base_url)

OllamaEmbeddings(model='bge-m3', base_url='http://104.59.217.59:11434\n', client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [4]:
llm = ChatOllama(model = 'gemma3:12b',
                 base_url=base_url)


In [ ]:
llm = ChatOllama(model = 'gemma3:12b',
                 base_url=base_url)


In [48]:
memory = ConversationBufferMemory(return_messages=True)

/tmp/ipykernel_1271325/3614923015.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [47]:
llm.invoke('hi').content

'Hi there! 😊 \n\nHow can I help you today?'

In [23]:
# Node 1: 질문 내용 구분 함수
def classify_question(stats):
    # input_question = {"question": "3 곱하기 7은 얼마야?"}

    question = stats["question"]
    messages = [
                    {"role": "system", 
                        "content": f"""당신은 질문이 '수학 문제'인지 아닌지를 분류하는 분류기입니다. 
                        수학 문제면 'math', 아니면 'general'로만 대답하세요.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ]
    response = llm.invoke(messages).content.strip().lower()
    stats['type']=response
    return stats

In [24]:
# Node 2: 수학 질문 처리
def answer_math(state):
    question = state["question"]
    messages = [
                    {"role": "system", 
                        "content": f"당신은 수학 문제만 푸는 계산기입니다."
                        },
                    {"role": "user", 
                    "content": question}
                ]
    answer = llm.invoke(messages).content
    state["answer"] = answer
    return state

In [25]:
# Node 3: 일반 질문 처리
def answer_general(state):
    question = state["question"]
    messages = [
                    {"role": "system", 
                        "content": f"당신은 다양한 질문에 답변하는 AI입니다."
                        },
                    {"role": "user", 
                    "content": question}
                ]
    answer = llm.invoke(messages).content
    state["answer"] = answer
    return state

In [30]:
builder = StateGraph(state_schema=state_type)

In [31]:
builder.add_node("classify", classify_question)
builder.add_node("math_mode", answer_math)
builder.add_node("general_mode", answer_general)

In [32]:
builder.set_entry_point("classify")

In [34]:
# 분기 조건 설정
def route_based_on_type(state):
    if state["type"] == "math":
        return "math_mode"
    else:
        return "general_mode"

In [35]:
builder.add_conditional_edges("classify", route_based_on_type)
builder.add_edge("math_mode", END)
builder.add_edge("general_mode", END)

In [36]:
# 그래프 컴파일
graph = builder.compile()


In [42]:
# 실행 예시
input_question = {"question": "근의 공식에 대해서 설명해주고 예시를 들어줄래?"}
result = graph.invoke(input_question)
print(result["answer"])

## 근의 공식 설명 및 예시

근의 공식은 2차 방정식의 해를 구하는 공식입니다. 2차 방정식은 일반적으로 다음과 같은 형태로 표현됩니다.

**ax² + bx + c = 0**

여기서 a, b, c는 상수이며, a는 0이 아니어야 합니다. 근의 공식은 이 2차 방정식의 해(x 값)를 다음과 같이 나타냅니다.

**x = (-b ± √(b² - 4ac)) / 2a**

**각 부분의 의미:**

*   **a, b, c:** 2차 방정식의 계수
*   **±:** 더하기 또는 빼기 (두 개의 해를 의미)
*   **√:** 제곱근
*   **b² - 4ac:** 판별식 (Discriminant)이라고 하며, 해의 종류를 판별하는 데 사용됩니다.

**판별식에 따른 해의 종류:**

*   **b² - 4ac > 0:** 서로 다른 두 실근을 갖습니다.
*   **b² - 4ac = 0:** 중근(실근)을 갖습니다.
*   **b² - 4ac < 0:** 서로 다른 두 허근을 갖습니다. (실수 범위에서는 해가 없습니다.)

**예시:**

다음 2차 방정식의 해를 근의 공식을 사용하여 구해보겠습니다.

**2x² + 5x - 3 = 0**

1.  **계수 확인:**
    *   a = 2
    *   b = 5
    *   c = -3

2.  **근의 공식 적용:**
    *   x = (-5 ± √(5² - 4 * 2 * -3)) / (2 * 2)
    *   x = (-5 ± √(25 + 24)) / 4
    *   x = (-5 ± √49) / 4
    *   x = (-5 ± 7) / 4

3.  **두 개의 해 계산:**
    *   x₁ = (-5 + 7) / 4 = 2 / 4 = 1/2
    *   x₂ = (-5 - 7) / 4 = -12 / 4 = -3

따라서, 2차 방정식 2x² + 5x - 3 = 0의 해는 x = 1/2 와 x = -3 입니다.

**다른 예시:**

1.  **x² - 4x + 4 = 0**
   

In [40]:
# 실행 예시
input_question = {"question": "인도와 인도네시아는 뭐가 달라?"}
result = graph.invoke(input_question)
print(result["answer"])

인도와 인도네시아는 지리적 위치, 문화, 역사 등 여러 면에서 차이가 있습니다. 두 나라의 주요 차이점을 자세히 살펴보겠습니다.

**1. 지리 및 위치:**

*   **인도:** 남아시아에 위치하며, 파키스탄, 중국, 네팔, 부탄, 방글라데시, 미얀마와 국경을 접하고 있습니다. 힌두스탄 반도에 위치하며, 다양한 지형을 가지고 있습니다.
*   **인도네시아:** 동남아시아와 태평양 사이에 위치한 섬나라로, 수많은 섬들로 이루어져 있습니다. 세계에서 가장 큰 군도 국가입니다.

**2. 인구 및 민족:**

*   **인도:** 세계에서 가장 인구가 많은 나라 중 하나이며, 다양한 민족과 언어가 공존합니다. 힌두교 신자가 다수를 차지합니다.
*   **인도네시아:** 세계에서 네 번째로 인구가 많은 나라이며, 다양한 민족과 언어가 존재합니다. 이슬람교 신자가 다수를 차지합니다.

**3. 문화:**

*   **인도:** 힌두교 문화의 영향을 많이 받았으며, 요가, 아유르베다, 인도 음악, 인도 영화 등 독특한 문화를 가지고 있습니다.
*   **인도네시아:** 다양한 문화적 영향을 받았으며, 인도, 중국, 아랍, 유럽 등의 문화가 혼합되어 있습니다. 전통 춤, 음악, 미술, 공예 등이 발달했습니다.

**4. 역사:**

*   **인도:** 고대 문명이 발달했으며, 불교, 자이나교 등 다양한 종교의 발상지입니다. 영국 식민지였다가 독립했습니다.
*   **인도네시아:** 다양한 왕국들이 번성했으며, 네덜란드 식민지였다가 독립했습니다.

**5. 경제:**

*   **인도:** 서비스업이 발달했으며, IT 산업이 빠르게 성장하고 있습니다.
*   **인도네시아:** 천연자원이 풍부하며, 농업, 광업, 제조업이 발달했습니다.

**6. 언어:**

*   **인도:** 힌디어, 영어, 그리고 다양한 지역 언어가 사용됩니다.
*   **인도네시아:** 인도네시아어를 사용하며, 지역적으로 다양한 언어가 사용됩니다.

**요약:**

| 특징 | 인도 | 인도네